In [1]:
# Set the current folder as root

import os

ROOT_PATH = "./ai-2025"

os.chdir(ROOT_PATH)

In [72]:
# Opening the data_set

import csv

DATA_SET = "credit.csv"

with open(DATA_SET, mode="r") as file:
    ds = [row for row in csv.DictReader(file)]

print(ds)

INDEX = "Indice"
target = "Risco"
DS_ATTR = [attr for attr in ds[0] if attr != target and attr != INDEX]

[{'Indice': 'E1', 'Historia de Crédito': 'Ruim', 'Dívida': 'Alta', 'Garantia': 'Nenhuma', 'Renda': '$0 a $15k', 'Risco': 'Alto'}, {'Indice': 'E2', 'Historia de Crédito': 'Desconhecida', 'Dívida': 'Alta', 'Garantia': 'Nenhuma', 'Renda': '$15 a $35k', 'Risco': 'Alto'}, {'Indice': 'E3', 'Historia de Crédito': 'Desconhecida', 'Dívida': 'Baixa', 'Garantia': 'Nenhuma', 'Renda': '$15 a $35k', 'Risco': 'Moderado'}, {'Indice': 'E4', 'Historia de Crédito': 'Desconhecida', 'Dívida': 'Baixa', 'Garantia': 'Nenhuma', 'Renda': '$0 a $15k', 'Risco': 'Alto'}, {'Indice': 'E5', 'Historia de Crédito': 'Desconhecida', 'Dívida': 'Baixa', 'Garantia': 'Nenhuma', 'Renda': 'Acima de $35k', 'Risco': 'Baixo'}, {'Indice': 'E6', 'Historia de Crédito': 'Desconhecida', 'Dívida': 'Baixa', 'Garantia': 'Adequada', 'Renda': 'Acima de $35k', 'Risco': 'Baixo'}, {'Indice': 'E7', 'Historia de Crédito': 'Ruim', 'Dívida': 'Baixa', 'Garantia': 'Nenhuma', 'Renda': '$0 a $15k', 'Risco': 'Alto'}, {'Indice': 'E8', 'Historia de Créd

In [3]:
import math

def calc_entropy(ds, target):
    tgt_column = [row[target] for row in ds]
    class_count = {}

    for tgt in tgt_column:
        class_count[tgt] = class_count.get(tgt, 0) + 1

    total = len(tgt_column)
    entropy = 0

    for count in class_count.values():
        prob = count / total
        entropy -= prob *  math.log(prob, 2)
        
    return entropy

class_entropy = calc_entropy(ds, target)

print(class_entropy)

1.5306189948485172


In [4]:
def calc_part_entropy(ds, attr, target):
    attr_entropy = {}
    attr_values = set(row[attr] for row in ds)

    for value in attr_values:
        part_attr_value = [row for row in ds if row[attr] == value]
        attr_entropy[value] = calc_entropy(part_attr_value, target)

    return attr_entropy

calc_part_entropy(ds, "wind", target)

KeyError: 'wind'

In [5]:
def calc_info_gain(ds, attr, base_entropy, part_entropy):
    total_count = len(ds)
    info_gain = base_entropy

    for part in part_entropy:
        count_attr = sum(1 for row in ds if row[attr] == part)
        prob = count_attr / total_count
        info_gain -= prob * part_entropy[part]

    return info_gain

calc_info_gain(ds, "wind", class_entropy, part_entropy)

NameError: name 'part_entropy' is not defined

In [6]:
ds_attr = [attr for attr in ds[0] if attr != target and attr != INDEX] 

attr_info_gain = {}

for attr in ds_attr:
    part_entropy = calc_part_entropy(ds, attr, target)
    attr_info_gain[attr] = calc_info_gain(ds, attr, class_entropy, part_entropy)

print(attr_info_gain)

{'Credito': 0.26565428452375395, 'Dívida': 0.06289889437401797, 'Garantia': 0.20604950908542197, 'Renda': 0.9663236712849368}


In [7]:
sunny_ds = [row for row in ds if row["outlook"] == "sunny"]
sunny_entropy = calc_entropy(sunny_ds, target)

ds_attr = [attr for attr in ds[0] if attr != target and attr != INDEX and attr != "outlook"] 

attr_info_gain = {}

for attr in ds_attr:
    part_entropy = calc_part_entropy(sunny_ds, attr, target)
    attr_info_gain[attr] = calc_info_gain(sunny_ds, attr, sunny_entropy, part_entropy)

print(attr_info_gain)

KeyError: 'outlook'

In [74]:
def id3(ds, target, dt=None):
    if dt is None:
        dt = {}
    
    class_entropy = calc_entropy(ds, target)

    ds_attr = [attr for attr in ds[0] if attr != target and attr != INDEX] 
    attr_info_gain = {}
    for attr in ds_attr:
        part_entropy = calc_part_entropy(ds, attr, target)
        attr_info_gain[attr] = calc_info_gain(ds, attr, class_entropy, part_entropy)

    biggest_info_gain = max(attr_info_gain, key=attr_info_gain.get)
    attr_values = set(row[biggest_info_gain] for row in ds)

    dt[biggest_info_gain] = {}
    
    for value in attr_values:
        value_ds = [row for row in ds if row[biggest_info_gain] == value]
        target_values = set([row[target] for row in value_ds])
        
        if len(target_values) == 1:
            dt[biggest_info_gain][value] = list(target_values)[0]
        else:
            dt[biggest_info_gain][value] = id3(value_ds, target)

    return dt

decision_tree = id3(ds, target)
print(decision_tree)
    

{'Renda': {'$15 a $35k': {'Dívida': {'Alta': {'Historia de Crédito': {'Desconhecida': 'Alto', 'Boa': 'Moderado', 'Ruim': 'Alto'}}, 'Baixa': {'Historia de Crédito': {'Desconhecida': 'Moderado', 'Boa': 'Baixo', 'Ruim': 'Moderado'}}}}, 'Acima de $35k': {'Historia de Crédito': {'Desconhecida': 'Baixo', 'Boa': 'Baixo', 'Ruim': 'Moderado'}}, '$0 a $15k': {'Historia de Crédito': {'Desconhecida': {'Dívida': {'Alta': 'Moderado', 'Baixa': {'Garantia': {'Adequada': 'Baixo', 'Nenhuma': 'Alto'}}}}, 'Boa': {'Dívida': {'Alta': 'Alto', 'Baixa': 'Baixo'}}, 'Ruim': 'Alto'}}}}


In [75]:
def print_tree(tree, prefix=""):
    if isinstance(tree, dict):
        keys = list(tree.keys())
        for i, key in enumerate(keys):
            # Choose the pointer based on whether it's the last key.
            pointer = "└── " if i == len(keys) - 1 else "├── "
            print(prefix + pointer + str(key))
            # Prepare the new prefix: add spacing if last, or a vertical bar if not.
            new_prefix = prefix + ("    " if i == len(keys) - 1 else "│   ")
            print_tree(tree[key], new_prefix)
    else:
        # Print the leaf node.
        print(prefix + "└── " + str(tree))

print_tree(decision_tree)

└── Renda
    ├── $15 a $35k
    │   └── Dívida
    │       ├── Alta
    │       │   └── Historia de Crédito
    │       │       ├── Desconhecida
    │       │       │   └── Alto
    │       │       ├── Boa
    │       │       │   └── Moderado
    │       │       └── Ruim
    │       │           └── Alto
    │       └── Baixa
    │           └── Historia de Crédito
    │               ├── Desconhecida
    │               │   └── Moderado
    │               ├── Boa
    │               │   └── Baixo
    │               └── Ruim
    │                   └── Moderado
    ├── Acima de $35k
    │   └── Historia de Crédito
    │       ├── Desconhecida
    │       │   └── Baixo
    │       ├── Boa
    │       │   └── Baixo
    │       └── Ruim
    │           └── Moderado
    └── $0 a $15k
        └── Historia de Crédito
            ├── Desconhecida
            │   └── Dívida
            │       ├── Alta
            │       │   └── Moderado
            │       └── Baixa
            │          